
--- Simple UPI App ---
Server status: LIVE
1. Check Balance
2. Send Money
3. Toggle Server (simulate down/up)
4. Exit


Enter choice:  4


Exiting app.


{'name': 'Bob', 'mobile': '9991110002', 'balance': 3000}


In [3]:
# ---------- Simple UPI App with AutoPay Queue + Frozen Amount + PIN ----------

# User data store (acting as a simple database)
# Each user has: name, mobile number, total balance, frozen balance, and PIN
users = [
    {"name": "Alice", "mobile": "9991110001", "balance": 5000, "frozen": 0, "pin": "1234"},
    {"name": "Bob", "mobile": "9991110002", "balance": 3000, "frozen": 0, "pin": "2345"},
    {"name": "Charlie", "mobile": "9991110003", "balance": 10000, "frozen": 0, "pin": "3456"},
    {"name": "David", "mobile": "9991110004", "balance": 2500, "frozen": 0, "pin": "4567"},
    {"name": "Eva", "mobile": "9991110005", "balance": 7000, "frozen": 0, "pin": "5678"},
]

# Helper function to get a user dictionary by their mobile number
def get_user(mobile):
    mobile = str(mobile)  # convert to string to handle int inputs too
    for u in users:
        if u["mobile"] == mobile:
            return u
    return None  # return None if not found

# Server status flag
server_down = False

# Pending payments queue (used when server is down)
# Each item in the queue is a dict: {"sender":..., "receiver":..., "amount":...}
pending_payments = []

# Check balance of a user (shows frozen balance also)
def check_balance(mobile):
    user = get_user(mobile)
    if user:
        # Available = total balance minus frozen
        available = user['balance'] - user['frozen']
        print(f"\n{user['name']}'s account:")
        print(f"  Total balance:  ₹{user['balance']}")
        print(f"  Frozen amount:  ₹{user['frozen']}")
        print(f"  Available now:  ₹{available}")
    else:
        print("User not found.")

# Process all pending (queued) AutoPay transactions when server is back up
def process_pending():
    """When server is back up, release frozen amounts and transfer money."""
    global pending_payments
    if not server_down and pending_payments:  # process only if server is live
        print("\nProcessing pending AutoPay transactions...")
        for tx in pending_payments:
            sender = get_user(tx["sender"])
            receiver = get_user(tx["receiver"])
            amount = tx["amount"]
            # Check again if frozen funds are still there
            if sender and receiver and sender["frozen"] >= amount:
                # Deduct frozen amount from sender
                sender["frozen"] -= amount
                # Deduct actual balance permanently
                sender["balance"] -= amount
                # Credit receiver
                receiver["balance"] += amount
                print(f"AutoPaid ₹{amount} from {sender['name']} to {receiver['name']}")
            else:
                print(f"AutoPay failed for {tx}")
        pending_payments = []  # clear queue after processing

# Send money from one user to another
def send_money(sender_mobile, receiver_mobile, amount):
    sender = get_user(sender_mobile)
    receiver = get_user(receiver_mobile)
    if not sender or not receiver:
        print("Sender or Receiver not found.")
        return

    # Ask for sender's PIN for security
    entered_pin = input(f"Enter {sender['name']}'s PIN: ")
    if entered_pin != sender["pin"]:
        print("Invalid PIN. Transaction cancelled.")
        return

    if server_down:
        # Server down: freeze the amount and queue it for AutoPay
        available = sender['balance'] - sender['frozen']
        if available >= amount:
            sender['frozen'] += amount  # freeze funds
            print("Server is down. Amount frozen and transaction queued for AutoPay once server is back.")
            pending_payments.append({
                "sender": sender_mobile,
                "receiver": receiver_mobile,
                "amount": amount
            })
        else:
            print("Insufficient available balance to freeze.")
    else:
        # Server live: transfer immediately
        available = sender['balance'] - sender['frozen']
        if available >= amount:
            sender['balance'] -= amount
            receiver['balance'] += amount
            print(f"₹{amount} sent from {sender['name']} to {receiver['name']}")
        else:
            print("Insufficient balance.")

# Menu-driven interface for the app
def menu():
    global server_down
    while True:
        print("\n--- Simple UPI App ---")
        print(f"Server status: {'DOWN' if server_down else 'LIVE'}")
        print("1. Check Balance")
        print("2. Send Money")
        print("3. Toggle Server (simulate down/up)")
        print("4. Exit")
        choice = input("Enter choice: ")

        if choice == "1":
            # Option 1: check balance
            mobile = input("Enter your mobile number: ")
            check_balance(mobile)

        elif choice == "2":
            # Option 2: send money
            sender = input("Sender mobile: ")    
            receiver = input("Receiver mobile: ")
            try:
                amount = float(input("Amount: "))
            except ValueError:
                print("Please enter a valid number.")
                continue
            send_money(sender, receiver, amount)

        elif choice == "3":
            # Option 3: toggle server state (simulate server down/up)
            server_down = not server_down
            print(f"Server status now: {'DOWN' if server_down else 'LIVE'}")
            # If server becomes live again, process pending payments automatically
            process_pending()

        elif choice == "4":
            # Option 4: exit app
            print("Exiting app.")
            break
        else:
            print("Invalid choice.")

# Entry point of the script
if __name__ == "__main__":
    menu()



--- Simple UPI App ---
Server status: LIVE
1. Check Balance
2. Send Money
3. Toggle Server (simulate down/up)
4. Exit


Enter choice:  3


Server status now: DOWN

--- Simple UPI App ---
Server status: DOWN
1. Check Balance
2. Send Money
3. Toggle Server (simulate down/up)
4. Exit


Enter choice:  2
Sender mobile:  9991110001
Receiver mobile:  9991110002
Amount:  200
Enter Alice's PIN:  1234


Server is down. Amount frozen and transaction queued for AutoPay once server is back.

--- Simple UPI App ---
Server status: DOWN
1. Check Balance
2. Send Money
3. Toggle Server (simulate down/up)
4. Exit


Enter choice:  1
Enter your mobile number:  9991110001



Alice's account:
  Total balance:  ₹5000
  Frozen amount:  ₹200.0
  Available now:  ₹4800.0

--- Simple UPI App ---
Server status: DOWN
1. Check Balance
2. Send Money
3. Toggle Server (simulate down/up)
4. Exit


Enter choice:  4


Exiting app.
